# 1- Drive Bağlantısı

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2- Kütüphaneler

In [2]:
!pip install pymap3d pyproj

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00


In [3]:
# 1. Matematik ve Sayısal İşlemler
import math
import numpy as np
import scipy.optimize # Optimizasyon işlemleri için
from scipy.spatial import distance # İki nokta arasındaki mesafeleri hesaplamak için.
from scipy import signal #  Sinyal işleme fonksiyonları için
from scipy.interpolate import InterpolatedUnivariateSpline # Veriler üzerinde spline interpolasyonu yapmak için. Yani; noktalar arasında düzgün, kesintisiz ve pürüzsüz eğriler oluşturur.

# 2. Görselleştirme ve Veri Analizi
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px # Etkileşimli ve daha gelişmiş görselleştirmeler için.
import plotly.graph_objects as go # Etkileşimli ve daha gelişmiş görselleştirmeler için.

# 3. Coğrafi Hesaplamalar
import pymap3d as pm # GNSS konum verilerinin dönüştürülmesi ve Vincenty formülleri ile mesafe hesaplamaları için.
import pymap3d.vincenty as pmv #GNSS konum verilerinin dönüştürülmesi ve Vincenty formülleri ile mesafe hesaplamaları için.
import pyproj as proj # Coğrafi projeksiyonlar ve dönüşümler için.

# 4. Makine Öğrenimi ve Derin Öğrenme
from tensorflow import keras # Derin öğrenme modelleri oluşturmak ve eğitmek için.
from keras import layers # Yapay sinir ağları katmanlarını tanımlamak ve oluşturmak için .
from keras import models # Yapay sinir ağları modellerini tanımlamak ve oluşturmak için.

# 5. Dosya ve İlerleme Takibi
import glob as gl # Dosya sistemi ile çalışmak ve belirli desenlere uyan dosyaları bulmak için.
from tqdm.auto import tqdm # Uzun süren işlemlerde  ilerleme çubuğu oluşturmak için.

# 6. Gereksiz Uyarıları Görmeme
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Sabitler
CLIGHT = 299_792_458   # Işık Hızı (m/s). GNSS sinyallerinin uydu ile alıcı arasında hareket etme hızını temsil edecek.
RE_WGS84 = 6_378_137   # Dünya Yarıçapı (WGS84 yöntemiyle ölçülmüştür) (m). Dünya'nın WGS84 referans elipsoidine göre ekvator yarıçapını temsil edecek.
OMGE = 7.2921151467E-5  # Dünya'nın Açısal Hızı (rad/s). Dünya'nın kendi ekseni etrafındaki dönüş hızını temsil eder.  Konum hesaplamalarında kullanılır.

# 3- Uydu Seçimi

Bu fonksiyon, bir GNSS veri çerçevesinden (ör. device_gnss.csv dosyasından alınan veriler) belirli kriterlere göre uydu sinyallerini seçmek için kullanılacaktır.

Fonksiyon, aşağıdaki ölçütlere uymayan uydu sinyallerini filtreleyerek daha güvenilir veriler elde edilmesini sağlar:
- **Taşıyıcı frekans hatası (CarrierErrorHz) 2 MHz'ten küçük olmalı.** Çünkü, GNSS sinyallerinde taşıyıcı frekans hatasının küçük olması, sinyalin doğru bir şekilde alınması için önemlidir. 2 MHz üzerindeki hatalar genellikle sinyaldeki parazit veya sapmalardan kaynaklanır ve güvenilir veri elde edilmesini zorlaştırır.

- **Uydu eğim açısı (SvElevationDegrees) 10 dereceden büyük olmalı.** Çünkü, 10 derece altındaki açılar veya 15 derece üstündeki açılar, sinyalin atmosferik etkilerden (örneğin troposferik kırılma ya da bir diğer deyişle GPS uydusunun sinyalinin varışındaki gecikme) daha fazla etkilenmesine neden olabilir. Ayrıca, alıcıya gelen sinyalin zayıf olması düşük eğim açılarında daha olasıdır.

- **Taşıyıcı-gürültü oranı (Cn0DbHz) 15 dB-Hz'den büyük olmalı.** Çünkü, Taşıyıcı-gürültü oranı, sinyal kalitesini gösteren önemli bir metriktir. 15 dB-Hz altındaki sinyaller zayıf olarak kabul edilir ve konum belirleme doğruluğunu olumsuz etkileyebilir.

- **Çoklu yol etkisi (MultipathIndicator) olmadığını belirtmeli.** Çünkü, Çoklu yol etkisi, GNSS sinyallerinin yüzeylerden yansıması sonucu farklı yollardan alıcıya ulaşmasıdır. Bu durum, sinyalin doğruluğunu ciddi şekilde bozabilir. Bu nedenle, yalnızca doğrudan gelen sinyaller seçilmelidir.

Bu fonksiyonu yazmamızdaki sebep, GNSS sinyallerinin doğruluğunu artırmak ve hatalı sinyallerden kaynaklanan sapmaları azaltmaktır.

In [4]:
# GNSS veri çerçevesinden uydu sinyallerini belirli kriterlere göre filtreleyen fonksiyon
def satellite_selection(df, column):
    """
    Bu fonksiyon, GNSS veri çerçevesinden belirli kriterlere uymayan uydu sinyallerini filtreler.

    Args:
        df : DataFrame from device_gnss.csv
            GNSS ölçümlerini içeren veri çerçevesi.
        column : str
            Sinyallerin filtreleneceği sütunun adı.

    Returns:
        df : DataFrame
            Belirtilen kriterlere uyan uydu sinyallerini içeren filtrelenmiş veri çerçevesi geri döndürülecek.
    """
    # Null olmayan değerlerle çalışmak için
    idx = df[column].notnull()

    # Taşıyıcı frekans hatası (CarrierErrorHz) 2 MHz'ten küçük olmalı. Çünkü:
    #  - GNSS sinyallerinde taşıyıcı frekans hatasının küçük olması, sinyali doğru bir şekilde almamız için önemlidir.
    #  - 2 MHz üzerindeki hatalar genellikle sinyaldeki parazit veya sapmalardan kaynaklanır ve güvenilir veriyi elde etmemizi zorlaştırır.
    idx &= df['CarrierErrorHz'] < 2.0e6

    # Uydu eğim açısı (SvElevationDegrees) 10 ila 15 derece arasında olmalı. Çünkü:
    #  - 10 derece altındaki açılar, sinyalin atmosferik etkilerden (örneğin troposferik kırılma - sinyaldeki gecikme) daha fazla etkilenmesine neden olabilir.
    #  - Ayrıca,  eğim açısı düştükçe alıcıya gelen sinyalin zayıflaması daha olasıdır.
    idx &= df['SvElevationDegrees'] > 10.0

    # Taşıyıcı-gürültü oranı (C/N0) 15 dB-Hz'den büyük olmalı. Çünkü:
    # - Taşıyıcı-gürültü oranı, sinyal kalitesini gösteren önemli bir metriktir.
    # - 15 dB-Hz altındaki sinyaller zayıf olarak kabul edilir ve konum belirleme doğruluğunu olumsuz etkileyebilir.
    idx &= df['Cn0DbHz'] > 15.0

    # Çoklu yol etkisi (MultipathIndicator = 0) olmamalı. Çünkü:
    # - Çoklu yol etkisi, GNSS sinyallerinin yüzeylerden yansıması sonucu farklı yollardan alıcıya ulaşmasıdır.
    # - Bu durum, sinyalin doğruluğunu ciddi şekilde bozabilir. Bu nedenle, yalnızca doğrudan gelen sinyaller seçilmeli , yüzeyden seken sinyalleri egale etmeliyiz.
    idx &= df['MultipathIndicator'] == 0

    # Belirtilen kriterlere uyan satırları döndür
    return df[idx]


# 4- Uydu Sinyallerinden Yeryüzündeki Bir Kullanıcının Konumunu ve Hızını Hesaplama (Weighted Least Squares / Ağırlıklı En Küçük Kareler Yönemi İle)

In [5]:
def point_positioning(gnss_df):
    # Her uydu (Svid) ve sinyal tipi (SignalType) için taşıyıcı frekansı (CarrierFrequencyHz) sütunundaki medyan değerleri hesaplıyoruz.
    CarrierFrequencyHzRef = gnss_df.groupby(['Svid', 'SignalType'])['CarrierFrequencyHz'].median()
    # Yukarıda oluştruduğumuz her medyan değerini, orijinal GNSS verileriyle birleştirdik
    gnss_df = gnss_df.merge(CarrierFrequencyHzRef, how='left', on=['Svid', 'SignalType'], suffixes=('', 'Ref'))

    # Üstte Hesapladığım CarrierFrequencyHzRef ile veri setinde bulunan CarrierFrequencyHz arasındaki farkı bularak taşıyıcı frekansı hatasını (CarrierErrorHz) hesapladık ...
    # ... ve veri setimizde 'CarrierErrorHz' sütununu ekledik
    # Çünkü başlarda da bahsettiğimiz gibi Taşıyıcı frekans hatası (CarrierErrorHz) 2 MHz'ten küçük olursa işimize yarayacak
    gnss_df['CarrierErrorHz'] = np.abs((gnss_df['CarrierFrequencyHz'] - gnss_df['CarrierFrequencyHzRef']))

    # Pseudorange verisini (gerçek mesafe ölçümünden elde edilen değer) Carrier Smoothing (taşıyıcı-pürüzsüzleştirme) yöntemiyle düzgünleştirdik.
    # Basitçe Carrier Smoothing: pseudorange verisinin doğruluğunu artırmak için kullanılan bir tekniktir. Verilerdeki gürültüyü azaltır ve daha doğru yer belirleme (positioning) sağlar
    gnss_df = carrier_smoothing(gnss_df)

    # Benzersiz zaman damgalarını alacak ve toplam iterasyon sayısını hesaplayacağız. Yani, kaç farklı zaman noktasının (epoch) veri setinde bulunduğunu göstereceğiz
    utcTimeMillis = gnss_df['utcTimeMillis'].unique() # "utcTimeMillis", her bir veri kaydının zamanını milisaniye cinsinden temsil ediyor.
    nepoch = len(utcTimeMillis)  # Toplam iterasyon sayısı

    # Pozisyon ve hız için başlangıç vektörlerini tanımladık
    x0 = np.zeros(4)  # Başlangıç pozisyon [x, y, z, saat] -> her birine ilk değer olarak 0 atadık
    v0 = np.zeros(4)  # Başlangıç hız [vx, vy, vz, saat] -> her birine ilk değer olarak 0 atadık

    # Pozisyon ve hız tahminlerini kayıt etmek için matrisler oluşturacağız
    x_wls = np.full([nepoch, 3], np.nan)  # Pozisyon tahminlerini kaydetmek için
    v_wls = np.full([nepoch, 3], np.nan)  # Hız tahminlerini kaydetmek için


    for i, (t_utc, df) in enumerate(tqdm(gnss_df.groupby('utcTimeMillis'), total=nepoch)):
        # Geçerli uydu sinyallerini Pseudorange verisi için seçelim
        df_pr = satellite_selection(df, 'pr_smooth')
        # Geçerli uydu sinyallerini Pseudorange Rate verisi için seçelim
        df_prr = satellite_selection(df, 'PseudorangeRateMetersPerSecond')  # PseudorangeRateMetersPerSecond, pseudorange'nin zamanla değişme hızını (yani, alıcı ile uydu arasındaki mesafenin hızını) ifade eder.

        # GNSS verisindeki pseudorange hatalarını düzeltmek için gerekli düzenlemeler yapacağız:
        # - SvClockBiasMeters: Uydu saatinin yanlışı (saat kayması)
        # - IsrbMeters:  uydunun yerçekimi, ortam koşulları veya diğer etmenler nedeniyle oluşan küçük sapmaları ifade eder
        # - IonosphericDelayMeters (İyonosferik Gecikme): GNSS sinyalleri, iyonosferdeki serbest elektron yoğunluğu nedeniyle hız kaybına uğrar.
        # - TroposphericDelayMeters (Troposferik Gecikme): Troposferdeki nem, sıcaklık ve basınç faktörler,, GNSS sinyallerini etkileyebilir ve bu da sinyallerin alıcıya ulaşma süresini uzatır.
        pr = (df_pr['pr_smooth'] + df_pr['SvClockBiasMeters'] - df_pr['IsrbMeters'] - df_pr['IonosphericDelayMeters'] - df_pr['TroposphericDelayMeters']).to_numpy()

        # Pseudorange Rate (Pseudorange Hızı) ve SvClock Drift (Uydu Saat Kayması) verilerini birleştirilerek, düzeltilmiş Pseudorange Rate elde edeceğiz.
        prr = (df_prr['PseudorangeRateMetersPerSecond'] + df_prr['SvClockDriftMetersPerSecond']).to_numpy()

        # Uydu pozisyon verilerini Pseudorange verisi için alacağız:
        #  - ECEF (Earth-Centered, Earth-Fixed): Dünya merkezine dayalı bir koordinat sistemidir.
        xsat_pr = df_pr[['SvPositionXEcefMeters', 'SvPositionYEcefMeters','SvPositionZEcefMeters']].to_numpy()

        # Uydu pozisyon verilerini Pseudorange Rate verisi için hız için alacağız
        xsat_prr = df_prr[['SvPositionXEcefMeters', 'SvPositionYEcefMeters','SvPositionZEcefMeters']].to_numpy()

        # Uydu hız verilerini Pseudorange Rate verisi için alacağız
        vsat = df_prr[['SvVelocityXEcefMetersPerSecond', 'SvVelocityYEcefMetersPerSecond','SvVelocityZEcefMetersPerSecond']].to_numpy()

        # Pseudorange ve Pseudorange Rate için ağırlık matrislerini hesaplayacağız
        # Buradaki "1 / df_pr['RawPseudorangeUncertaintyMeters']" işlemi:
        # Her bir belirsizliğin tersini alır. Belirsizliğin tersten alınması, daha düşük belirsizliğe sahip ölçümlere daha fazla ağırlık verir...
        # ... Yüksek belirsizlik, düşük ağırlık anlamına gelir. Yani, güvenilir olmayan ölçümler daha düşük bir ağırlığa sahip olmalı.
        Wx = np.diag(1 / df_pr['RawPseudorangeUncertaintyMeters'].to_numpy())
        Wv = np.diag(1 / df_prr['PseudorangeRateUncertaintyMetersPerSecond'].to_numpy())


        # Artık kullanıcının POZİSYON TAHMİNİNİ yapabiliriz

        if len(df_pr) >= 4:  # Konumu doğru belirleyebilmek için en az 4 uydu gerekir -> 3D konum + zaman
            if np.all(x0 == 0):  # Başlangıç tahmini 0 ise
                # Bu kısımda, LEAST SQUARES (en küçük kareler) yöntemi kullanılarak pozisyon tahmini yapacağız:
                # - pr_residuals: Bu fonksiyon, modelle uydu ölçümleri arasındaki farkları hesaplayacak. (residuals = artık)
                # - jac_pr_residuals: ölçüm hatalarının (residul) türevini alarak doğrusal olmayan optimizasyonun yapılmasını sağlar.
                # - xsat_pr: Uyduların konum verilerini içeren NumPy dizisi.
                # - pr: Pseudorange (mesafe ölçüm) verileri.
                # - Wx: Pseudorange belirsizliklerine dayalı ağırlık matrisi.
                opt = scipy.optimize.least_squares(pr_residuals, x0, jac_pr_residuals, args=(xsat_pr, pr, Wx))

                # Daha sonra yukarıda optimize edilen pozisyonu güncel pozisyonumuz olarak güncelledik
                x0 = opt.x

            # Normal WLS'ten sonra Robust (dayanıklı) WLS ile pozisyon tahmini yaparak ölçüm doğruluğunu arttıracağız:
            # Burada" soft_l1" loss fonksiyonu kullanılarak, pozisyon tahmini yapılırken daha az güvenilir ölçümlerin etkisi azaltılmak istiyoruz.
            opt = scipy.optimize.least_squares(pr_residuals, x0, jac_pr_residuals, args=(xsat_pr, pr, Wx), loss='soft_l1')


            # Optimizasyonun durumunu kontrol ederek başarılı mı başarısız mı diye bakalım
            # - (0) = Başarılı optimizasyon
            # - (1) = Başarıyla tamamlanmış ancak istenen hassasiyete ulaşılmamış.
            # - (2) = Optimizasyon başarıyla tamamlanmış ancak maksimum iterasyon sayısına ulaşılmış. Yani, çözüme ulaşmak için daha fazla iterasyon yapılabilirdi ancak durdurulmuş.
            # - (-1): Başarısızlık, optimizasyonun çözüme ulaşamadığı durum.
            # - (-2): Başarısızlık.
            if opt.status < 1 or opt.status == 2:
                print(f'i = {i} KONUM optimizasyon durumu = {opt.status}')

            else:
              # Eğer optimizasyon başarılı bir şekilde tamamlanmışsa, tahmin edilen pozisyon (opt.x) alınır ve x_wls'ye kaydedilir.
              # Buradaki opt.x[:3] kısmı, elde edilen 4 elemanlı çözüm vektörünün (3D + Zaman) ilk 3 elemanını (x, y, z koordinatlarını) alır.
                x_wls[i, :] = opt.x[:3]  # Tahmin edilen pozisyonu kaydet
                x0 = opt.x  # Pozisyonu güncelle




        # Kullanıcının HIZ TAHMİNİNİ de yapabliriz

        # Üstte konum için yaptığımız tüm işlemlerin aynısını hız için de yapacağız
        if len(df_prr) >= 4:  # En az 4 uydu verisi gerekli demiştik
            if np.all(v0 == 0):  # Başlangıç tahmini yoksa normal WLS ile başla
                opt = scipy.optimize.least_squares(prr_residuals, v0, jac_prr_residuals, args=(vsat, prr, x0, xsat_prr, Wv))
                v0 = opt.x  # Optimize edilen hızı güncelledik
            # Robust (Dayanıklı) WLS ile hız tahmini yaptık
            opt = scipy.optimize.least_squares(prr_residuals, v0, jac_prr_residuals, args=(vsat, prr, x0, xsat_prr, Wv), loss='soft_l1')
            if opt.status < 1:  # Yİne optimizaston durumlarını kontrol edeceğiz
                print(f'i = {i} HIZ optimizasyon durumu = {opt.status}')
            else:
                v_wls[i, :] = opt.x[:3]  # Tahmin edilen hızı kaydet
                v0 = opt.x  # Hızı güncelle

    # Tahmin edilen zaman, pozisyon ve hız değerlerini döndür
    return utcTimeMillis, x_wls, v_wls


# 5- Carrier Smoothing (Taşıyıcı Pürüzsüzleştirme) ile Pseudorange (Gerçek Mesafa , Konum/Hız Tahmini) Tahmininin Doğruluğunu Artırma

Bu fonksiyon, GNSS verilerinde taşıyıcı-faz ve pseudorange ölçümlerini birleştirerek taşıyıcı pürüzsüzleştirme (carrier smoothing) yöntemiyle daha hassas bir pseudorange tahmini yapar. Bu işlem, sinyal hatalarını ve sapmaları azaltmayı hedefler.

Pseudorange tahmini, GNSS (Global Navigation Satellite System) verilerinden kullanıcı konumunu ve hızını belirlemek için temel bir girdidir. Bu tahminler, uydu ile alıcı arasındaki ölçülen mesafeyi temsil eder, ancak bu mesafeler çeşitli hatalardan (örneğin, iyonosferik ve troposferik gecikmeler, saat hataları) etkilenir. **Carrier smoothing, bu hataları azaltarak daha güvenilir bir pseudorange tahmini sağlar.**

In [6]:
def carrier_smoothing(gnss_df):
    """
    Returns:
        df:'pr_smooth' adında taşıyıcı pürüzsüzleştirilmiş pseudorange DataFram'i

    """

    # Eşik Değerlerimizi tanımlayalım:

      # Taşıyıcı faz sıçrama eşiği [metre].
      # Bu değer, taşıyıcı faz ölçümlerindeki ani sıçramaların (cycle slip) tolerans sınırını belirler.
    carr_th = 1.6

      # Pseudorange sıçrama eşiği [metre].
      # Bu değer, pseudorange ölçümlerinde ani değişimlerin hata olarak algılanması için sınır belirler.
    pr_th =  20.0



    # pseudorange değerlerini taşıyıcı-pürüzsüzleştirme (carrier smoothing) işlemi sonrası saklayacak bir dizi oluşturduk
    prsmooth = np.full_like(gnss_df['RawPseudorangeMeters'], np.nan)  # np.nan ile birlikte pürüzsüzleştirme işlemi sonrası değerleri yer değiştireceğiz


    # Her bir sinyal için taşıyıcı-pürüzsüzleştirme (carrier smoothing) işlemini gerçekleştireceğiz:
    for (i, (svid_sigtype, df)) in enumerate((gnss_df.groupby(['Svid', 'SignalType']))):

        # 'AccumulatedDeltaRangeMeters' (Birikmiş Delta Menzil Ölçerler) sütunundaki 0 değerlerini NaN (geçersiz) ile değiştiriyoruz...
        # ... Böylece hatalı verilerin işlemi etkilemesini engelleyeceğiz.
        df = df.replace({'AccumulatedDeltaRangeMeters': {0: np.nan}})


        # Doppler hız bilgisiyle pseudorange (drng2) ve taşıyıcı faz (drng1) değişimlerini karşılaştıracağız...
        # ... Böylece, döngü kaymaları (cycle-slip) gibi hataları tespit edeceğiz.
        drng1 = df['AccumulatedDeltaRangeMeters'].diff() - df['PseudorangeRateMetersPerSecond']
        drng2 = df['RawPseudorangeMeters'].diff() - df['PseudorangeRateMetersPerSecond']


        # Farklı durumlara göre döngü kayması (cycle-slip) hatalarını kontrol etmeliyiz. Yani 'hata türlerini' belirlemeliyiz:
          # - slip3: Taşıyıcı faz değişimi belirlediğimiz eşik değerini (carr_th) aşarsa hata olarak işaretlenecek.
          # - slip4: Pseudorange değişimi belirlediğimiz eşik değerini (pr_th) aşarsa hata olarak işaretlenecek.
        slip1 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**1) != 0  # sıfırlama bayrağı
        slip2 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**2) != 0  # döngü kayma bayrağı
        slip3 = np.fabs(drng1.to_numpy()) > carr_th
        slip4 = np.fabs(drng2.to_numpy()) > pr_th


        # Üstte oluşturduğumuz döngü hata türlerini bulduğumuzda bunları işaretleyeceğiz:
        idx_slip = slip1 | slip2 | slip3 | slip4 # Herhangi bir döngü kayması durumu 'True' olarak işaretlenecek. Bunun için üstte belirlediğimiz hata türlerini bir değişkende tuttuk.
        # İlk veri noktasını True olarak işaretleyeceğiz. Çünkü ilk veri noktasından önce bir veri olmadığı için döngü kayması içeriyor gini değerlendirdik.
        idx_slip[0] = True



        # Sürekli taşıyıcı faz izleme gruplarını oluştur
        df['group_slip'] = np.cumsum(idx_slip)  # Her döngü kayması tespit edildiğinde grup numarası bir artırılır.




        # Pseudorange (ham mesafe ölçümü) ile taşıyıcı faz ölçümü arasındaki farkı hesapladık
        # Bu fark [dpc], ölçüm hatalarının analizinde ve taşıyıcı faz ölçümlerine dayalı doğruluğun artırılmasında önemli bir rol oynayacak
        df['dpc'] = df['RawPseudorangeMeters'] - df['AccumulatedDeltaRangeMeters']




        # Üstte oluşturduğumuz group_slip'e göre her gruptaki dpc(Psudorange - carrier phase) değerlerinin  ortalamasını hesapladık
        meandpc = df.groupby('group_slip')['dpc'].mean()
        df = df.merge(meandpc, on='group_slip', suffixes=('', '_Mean')) # Bu ortalamaları df'e ekledik



        # Hata içeren uydu id'lerini ve sinyal tiplerini orijinal gnns_df'te buluyoruz
        idx = (gnss_df['Svid'] == svid_sigtype[0]) & (gnss_df['SignalType'] == svid_sigtype[1])



        # Bu gata içeeren uydulara taşıyıcı faz ölçümü ve bias'ı ekleyerek düzgünleştirilmiş pseudorange'ı hesaplıyoruz
        prsmooth[idx] = df['AccumulatedDeltaRangeMeters'] + df['dpc_Mean']



    # Eğer taşıyıcı-pürüzsüzleştirme mümkün değilse, orijinal pseudorange değerini kullanacağız:
    idx_nan = np.isnan(prsmooth) # NaN değerlerini bul
    prsmooth[idx_nan] = gnss_df['RawPseudorangeMeters'][idx_nan]  # NaN olan yerlerde orijinal pseudorange'ı kullan
    gnss_df['pr_smooth'] = prsmooth # Sonuçları 'pr_smooth' sütununa ata

    return gnss_df

# 6- Pozisyon Tahmin ve Artıklık Hesaplamaları (Psödorange/Doppler Kalıntıları ve Jacobian Matrisi)
Bu kısımda yazacağımız 5 fonksiyon kullanıcı ve uydu arasındaki pozisyon, hız ve diğer etkenleri dikkate alarak GNSS verilerinden doğruluğu artıran hesaplamalar yapacaktır.

Residuals = Kalıntı , Artık , Sapma

In [7]:
# los_vector(xusr, xsat) fonksiyonu, kullanıcı (xusr) ile uydu (xsat) arasındaki doğrultu vektörünü (line-of-sight vector) ve mesafeyi hesaplayacak.
# GNSS konumlandırma sistemlerinde temel bir fonksiyondur çünkü doğru bir pozisyon tahmini için doğrultu ve mesafe bilgisi gereklidir.
def los_vector(xusr, xsat):
    """
    Args:
        xusr : ECEF'teki kullanıcı konumu (m) (ECEF = Earth Centered Earth Fixed Koordinat Sistemi)
        xsat : ECEF'teki uydu konumu (m)
    Returns:
        u: ECEF'teki kullanıcıdan uyduya doğru olan birim doğrultu vektörü (unit line-of-sight)  (m)
        rng: Kullanıcı ve uydu arasındaki mesafe (m)
    """
    u = xsat - xusr # doğrultunun yönü
    rng = np.linalg.norm(u, axis=1).reshape(-1, 1)  # Kullanıcı ve uydu arasındaki mesafe
    u /= rng

    return u, rng.reshape(-1)





# Jacobian matrisini hesaplayan fonksiyonumuz
# Bu matris, konum tahmin hatalarını minimize etmek için kullanılan bir optimizasyon yöntemi.
def jac_pr_residuals(x, xsat, pr, W):
    """
    Args:
        x : ECEF'teki mevcut pozisyon (m)
        xsat : ECEF'te uydu pozisyonu (m)
        pr : pseudorange (m)
        W : ağırlık matrisi
    Returns:
        W*J : Jacobian matrisi
    """
    u, _ = los_vector(x[:3], xsat) # Kullanıcı ve uydu arasındaki doğrultu vektörünü hesaplayacak. Buradaki 'u, _' işlemi =  (paketten çıkarma) özelliğidir. Fonksiyondan dönen birden fazla değeri birden fazla değişkene atamak için kullanılır.
                                   # Kullanıcının ilk 3 koordinatını alacak
    J = np.hstack([-u, np.ones([len(pr), 1])])   # Jacobian matrisi: [-ux -uy -uz 1]

    return W @ J  # Ağırlıklı Jacobian matrisi, yani W ile çarpılmış J matrisini bize döndürecek






# Pseudorange artıklarını hesaplayan fonksiyonumuz.
# Yani, kullanıcı pozisyonu ile uydu arasındaki mesafe farkı ile ölçülen pseudorange (gerçek mesafe) arasındaki fark
def pr_residuals(x, xsat, pr, W):
    """
    Args:
        x : kullanıcının ECEF'teki mevcut pozisyon (m)
        xsat : ECEF'te uydu pozisyonu (m)
        pr : pseudorange (m)
        W : ağırlık matrisi
    Returns:
        residuals*W : pseudorange hataları , artıkları
    """
    u, rng = los_vector(x[:3], xsat)  # Üstte yazdığımıuz los_vector fonksiyonu çağırarak  kullanıcı ile uydu arasındaki doğrultu vektörünü (u) ve mesafeyi (rng) hesaplıyoruz.

    # ! Mesafeyi hesaplarken Dünya'nın dönüşünden kaynaklanan 'Sagnac Etkisi'ni dikkate almalıyız.
    # Burada, OMGE Dünya'nın açısal hızı (radyan/saniye) ve CLIGHT ışık hızı (metre/saniye) sabitlerini kullanılarak bir düzeltme uyguladık.
    rng += OMGE * (xsat[:, 0] * x[1] - xsat[:, 1] * x[0]) / CLIGHT

    # Add GPS L1 clock offset
    residuals = rng - (pr - x[3]) # Gerçek mesafe ile ölçülen pseudorange arasındaki fark yani artık
                                  # 'rng': Hesaplanan gerçek mesafe , 'pr - x[3]': Ölçülen pseudorange değeri.
                                  # x[3] = zaman bileşenini temsil ediyor (x,y,z ve zaman'dan zaman'ı aldık)

    return residuals @ W   # Artıkları ağırlık matrisi ile çarpıp bize geri döndürecek





# Bu fonksiyon,  kullanıcının ve uydunun hızları ve pozisyonlarına dayanarak pseudorange rate (PRR)'i için Jacobian matrisini hesaplayacak.
# Bu da pozisyon ve hız tahmininin doğruluğunu optimize etmekte kullanılacak
def jac_prr_residuals(v, vsat, prr, x, xsat, W):
    """
    Args:
        v : Kullanıcının mevcut hızı, ECEF koordinat sisteminde (m/s).
        vsat : Uydu hızları, ECEF koordinat sisteminde (m/s).
        prr : Pseudorange rate (m/s), uydu ile kullanıcı arasındaki mesafe değişim hızıdır.
        x : Kullanıcının mevcut pozisyonu, ECEF koordinat sisteminde (m).
        xsat : Uydunun pozisyonu, ECEF koordinat sisteminde (m).
        W : Ağırlık matrisi
    Returns:
        W*J : Jacobian matrisi
    """
    u, _ = los_vector(x[:3], xsat) # 2. fonksitonda da yaptığımız kullanıcı ve uydu arasındaki doğrultu vektörünü hesaplayan işlem.
    J = np.hstack([-u, np.ones([len(prr), 1])])  # Jacobian matrisi [-ux -uy -uz 1] şeklinde olmalı

    return np.dot(W, J)  # W matrisi ile J matrisini çarpımını bize döndür






# Pseudorange rate artıklarını hesaplyacağız
def prr_residuals(v, vsat, prr, x, xsat, W):
    """
    Args:
        v : Kullanıcının mevcut hızı, ECEF koordinat sisteminde (m/s).
        vsat : Uydu hızları, ECEF koordinat sisteminde (m/s).
        prr : Pseudorange rate (m/s), uydu ile kullanıcı arasındaki mesafe değişim hızıdır.
        x : Kullanıcının mevcut pozisyonu, ECEF koordinat sisteminde (m).
        xsat : Uydunun pozisyonu, ECEF koordinat sisteminde (m).
        W : Ağırlık matrisi
    Returns:
        residuals*W : pseudorange rate artıkları , hataları
    """
    u, rng = los_vector(x[:3], xsat)

    # Pseudorange rate hesaplaması yapacağız:
      # İlk olarak uydu hızından kullanıcı hızını çıkararak doğrultu vektörü (u) ile çarpıp toplamını alıyoruz
      # Burda da yine aynı şekilde Dünya dönmesinin etkisini 'Sagnac Etkisi'ni göz önünde bulunduruyoruz
    rate = np.sum((vsat - v[:3]) * u, axis=1) + OMGE / CLIGHT * (vsat[:, 1] * x[0] + xsat[:, 1] * v[0] - vsat[:, 0] * x[1] - xsat[:, 0] * v[1])

    # Rate'i bulduğumuza göre artıkları (prr ile tahmin edilen hız arasındaki fark) hesaplayabiliriz
    residuals = rate - (prr - v[3])

    return residuals @ W

In [ ]:
# Simple outlier detection and interpolation
def exclude_interpolate_outlier(x_wls, v_wls):


In [ ]:
# Kalman filter
def Kalman_filter(zs, us, phone):


In [8]:
path = '/content/drive/MyDrive/BitirmeProjesi/smartphone-decimeter-2022/train/2021-07-19-US-MTV-1/GooglePixel4'
drive, phone = path.split('/')[-2:]

# Read data
gnss_df = pd.read_csv(f'{path}/device_gnss.csv')  # GNSS data
gt_df = pd.read_csv(f'{path}/ground_truth.csv')  # ground truth

# Point positioning
utc, x_wls, v_wls = point_positioning(gnss_df)

# Exclude velocity outliers
x_wls, v_wls = exclude_interpolate_outlier(x_wls, v_wls)

# Kalman smoothing
x_kf, _, _ = Kalman_smoothing(x_wls, v_wls, phone)

# Convert to latitude and longitude
llh_wls = np.array(pm.ecef2geodetic(x_wls[:, 0], x_wls[:, 1], x_wls[:, 2])).T
llh_kf = np.array(pm.ecef2geodetic(x_kf[:, 0], x_kf[:, 1], x_kf[:, 2])).T

# Baseline
x_bl = gnss_df.groupby('TimeNanos')[
    ['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']].mean().to_numpy()
llh_bl = np.array(pm.ecef2geodetic(x_bl[:, 0], x_bl[:, 1], x_bl[:, 2])).T

  0%|          | 0/1896 [00:00<?, ?it/s]

NameError: name 'exclude_interpolate_outlier' is not defined